<a href="https://colab.research.google.com/github/AnastasiyaKrokhina/lantibiotics/blob/main/Lantibiotic_Aziz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install biopython

In [5]:
import pandas as pd
from Bio import SeqIO
import requests
from io import StringIO

In [6]:
# чтение урл адреса к файлу
# выполняется GET-запрос по указанной ссылке, и возвращаемый ответ сервера сохраняется в переменную data в виде текстовой строки.
link = "https://raw.githubusercontent.com/AnastasiyaKrokhina/lantibiotics/main/data/Lantibiotics_2021-09.fasta"
data = requests.get(link).text

список переменных `bees_var`, каждая из которых содержит данные из отдельного файла, расположенного по определенному URL-адресу в Интернете. Файлы имеют расширение .tsv и содержат данные о совпадениях белков у пчел. Параметр `sep='\t'` указывает, что данные в файлах разделены символом табуляции. `header=None` указывает, что в файлах нет заголовка столбцов.

Список `bees_names` содержит названия файлов, соответствующие каждой переменной в списке `bees_var`.

In [7]:
# чтение файла
bees_var = [pd.read_csv(f'https://raw.githubusercontent.com/AnastasiyaKrokhina/lantibiotics/main/data/bee1/bee_{i}_matches.tsv', sep='\t', header=None) for i in [13, 20, 25, 33, 41, 4, 66, 75]]
bees_names = ['bee_13', 'bee_20', 'bee_25', 'bee_33', 'bee_41', 'bee_4', 'bee_66', 'bee_75']

Переменная `fasta_iterator` создается с помощью функции `SeqIO.parse()`, которая принимает два аргумента: `data` - строку с данными в формате FASTA, и `"fasta"` - указание на формат данных.

Затем из объектов последовательностей, полученных с помощью `fasta_iterator`, извлекаются идентификаторы лантибиотиков в переменную `lantibiotics_id_list`

In [8]:
#парсинг
fasta_iterator = SeqIO.parse(StringIO(data), "fasta")
# Extract the lantibiotics IDs
lantibiotics_id_list = [seq.id for seq in fasta_iterator]

In [9]:
# создание датафрейма
df = pd.DataFrame({ #'№': range(1, len(lantibiotics_id_list) + 1),
                   'Лантибиотик': lantibiotics_id_list})

Цикл `for` используется для итерации по двум спискам `bees_var` и `bees_names`, которые должны быть одинаковой длины.

Внутри цикла создается новая переменная `bee`, которой присваивается значение текущего элемента из списка `bees_var`. Затем выбираются первые три столбца из датафрейма `bee` и сохраняются в переменной `bee`.

Далее, метод `value_counts()` вызывается на столбце с индексом 1 в датафрейме `bee` и результат сохраняется в переменной `lantibiotics`.

In [10]:
# заполнение датафрейма
for bee, name in zip(bees_var, bees_names):
    # заполнение датафрейма
    bee = bee.iloc[:, :3]
    lantibiotics = bee[1].value_counts()

In [11]:
    # заполнение датафрейма
    df[name] = df['Лантибиотик'].apply(lambda x: lantibiotics.get(x, 0) / len(bee) * 100)
    print(df)

         Лантибиотик  bee_75
0     WP_029376348.1     0.0
1     WP_042739435.1     0.0
2     WP_015057295.1     0.0
3     WP_023416956.1     0.0
4     WP_155961898.1     0.0
...              ...     ...
3562      EHD55949.1     0.0
3563      EKU40925.1     0.0
3564      EFB97951.1     0.0
3565          JC6527     0.0
3566      EFI65092.1     0.0

[3567 rows x 2 columns]


In [12]:
# удалить строки в таблице, где все значения равны 0%.
df = df.loc[(df[df.columns[3:]] != 0).any(axis=1)]

print(df)

Empty DataFrame
Columns: [Лантибиотик, bee_75]
Index: []
